In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
!ls -1 *.csv

indicator gapminder population.csv
indicator gdp_per_capita_ppp version8.csv
indicator life_expectancy_at_birth.csv


In [3]:
def read_and_dropna(filename, index_column_name):
    df = pd.read_csv(filename)
    df = df[df[index_column_name].apply(lambda x: not pd.isnull(x))]
    return df.set_index(index_column_name)

def fill_na_forward_or_zero(df):
    return df.fillna(method='ffill', axis=1).fillna(0)

def convert_numeric(df):
    return df.applymap(lambda x: float(str(x).replace(',', '')))

life = read_and_dropna('indicator life_expectancy_at_birth.csv', 'Life expectancy')
gdp_pc = read_and_dropna('indicator gdp_per_capita_ppp version8.csv', 'income per person')
population = read_and_dropna('indicator gapminder population.csv', 'Total population')

life = convert_numeric(fill_na_forward_or_zero(life))
gdp_pc = convert_numeric(fill_na_forward_or_zero(gdp_pc))
population = convert_numeric(fill_na_forward_or_zero(population))

In [4]:
#df = pd.read_csv('indicator life_expectancy_at_birth.csv')
#df['Life expectancy'].apply(lambda x: not pd.isnull(x))

In [5]:
#life.fillna(0)

In [6]:
len(population)
type(life.ix[0][0])
np.isnan(np.NAN)
np.isnan(np.NaN)
np.isnan(2)
np.isnan(life.ix[0][0])
#help(np.isnan)
pd.isnull(3)
pd.isnull(life.ix[0][0])

/Users/hupili/Projects/DEMP2017/dmep2017.github.io/venv/lib/python3.5/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


False

In [7]:
#life.ix[['Abkhazia', 'Afghanistan']]
df = pd.DataFrame([
    [1, 2, 3],
    [4, np.NaN, np.NaN],
    [5, np.NaN, np.NaN]
])
df.fillna(method='ffill')
df.fillna(method='ffill', axis=1)
#life.columns

,0,1,2
0,1.0,2.0,3.0
1,4.0,4.0,4.0
2,5.0,5.0,5.0


In [8]:
len(life)

260

In [9]:
# Filter out the rows with missing values
life = life.ix[life.min(axis=1) > 0.01]
len(life)

201

In [10]:
all_countries = set(life.index) & set(gdp_pc.index) & set(population.index)
len(all_countries)

190

In [11]:
def slice_by_year(year):
    year = str(year)
    df = pd.concat([
        life.ix[all_countries][year],
        gdp_pc.ix[all_countries][year],
        population.ix[all_countries][year],
    ], axis=1)
    df.columns = ['life', 'gdp_pc', 'population']
    #return df
    df = df.fillna(0)
    df = df.sort_values(by='gdp_pc', ascending=False)
    # Visualization purpose: Able to apply log scale on this attribute
    df['population'] = df['population'] + 1 
    return df[:20]

In [12]:
#population['1800'] + 1

In [13]:
all_years = set(life.columns) & set(gdp_pc.columns) & set(population.columns)
all_years = sorted(all_years)
#all_years

In [14]:
all_data = [(
    year, 
    list(slice_by_year(year).T.to_dict().items())
) for year in all_years]

/Users/hupili/Projects/DEMP2017/dmep2017.github.io/venv/lib/python3.5/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.
/Users/hupili/Projects/DEMP2017/dmep2017.github.io/venv/lib/python3.5/site-packages/ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """
/Users/hupili/Projects/DEMP2017/dmep2017.github.io/venv/lib/python3.5/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-

In [15]:
open('all_data.js', 'w').write('allData=%s' % json.dumps(all_data))

122218

In [16]:
def get_values(d, field):
    values = []
    for (y, yd) in d:
        for (c, cd) in yd:
            values.append(cd.get(field, None))
    return values

life_values = get_values(all_data, 'life')
gdp_pc_values = get_values(all_data, 'gdp_pc')
population_values = get_values(all_data, 'population')

In [17]:
boundary_variables = """
lifeDomain = [%s, %s]
gdpDomain = [%s, %s]
populationDomain = [%s, %s]
""" % (
    min(life_values), 
    max(life_values),
    min(gdp_pc_values), 
    max(gdp_pc_values),
    min(population_values),
    max(population_values)
)

open('all_data.js', 'a').write('\n%s' % boundary_variables)
print(boundary_variables)


lifeDomain = [25.4, 82.66]
gdpDomain = [1284.40935, 119849.2934]
populationDomain = [27351.0, 307231962.0]



In [18]:
#life_values

In [19]:
#population.max(axis=1)
#len(population)
#population

In [20]:
#gdp_pc.max()

In [21]:
#population[all_years].ix['Afghanistan']
#population.fillna(0).applymap(lambda x: int(str(x).replace(',', ''))).max().max()